# Current Blockchain ( 2nodes )

In [ ]:
!rm -f /tmp/*.json
!rm -f /tmp/*.pb

In [ ]:
!peer lifecycle chaincode querycommitted --channelID mychannel --name scray-invoice-example --output json

# init peer access (certificates,...)

In [1]:
import os
import json
import subprocess
import pandas as pd
 
pwd0 = "/home/jovyan/work/fabric-samples/"
pwd = "/home/jovyan/work/fabric-samples/test-network"

os.environ['PATH'] = "/home/jovyan/work/fabric-samples/bin:" + str(os.environ.get('PATH')) 
os.environ['CORE_PEER_TLS_ENABLED'] = "true"
os.environ['CORE_PEER_LOCALMSPID'] = "Org1MSP"
os.environ['CORE_PEER_TLS_ROOTCERT_FILE'] = pwd + "/organizations/peerOrganizations/org1.example.com/peers/peer0.org1.example.com/tls/ca.crt"
os.environ['CORE_PEER_MSPCONFIGPATH'] = pwd + "/organizations/peerOrganizations/org1.example.com/users/Admin@org1.example.com/msp"
os.environ['CORE_PEER_ADDRESS'] = "peer0.org1.example.com:7051"
os.environ['FABRIC_CFG_PATH'] = pwd0 + "/config/"

addr0      = 'peer0.org1.example.com:7050'
orderer    = 'orderer.example.com'
ordererpem = pwd + '/organizations/ordererOrganizations/example.com/orderers/orderer.example.com/msp/tlscacerts/tlsca.example.com-cert.pem'

addr1  = 'peer0.org1.example.com:7051'
crt1   = pwd + '/organizations/peerOrganizations/org1.example.com/peers/peer0.org1.example.com/tls/ca.crt'
msp1   = pwd + '/organizations/peerOrganizations/org1.example.com/users/Admin@org1.example.com/msp'
mspid1 = "Org1MSP"

addr2  = 'peer0.org2.example.com:9051'
crt2   = pwd + '/organizations/peerOrganizations/org2.example.com/peers/peer0.org2.example.com/tls/ca.crt' 
msp2   = pwd + '/organizations/peerOrganizations/org2.example.com/users/Admin@org2.example.com/msp'
mspid2 = "Org2MSP"

def createEnvironmentForPeer(localmspid=mspid1, tlsrootcert=crt1, mspconfigpath=msp1, peeraddress=addr1):
    os.environ['CORE_PEER_LOCALMSPID']        = localmspid
    os.environ['CORE_PEER_TLS_ROOTCERT_FILE'] = tlsrootcert
    os.environ['CORE_PEER_MSPCONFIGPATH']     = mspconfigpath
    os.environ['CORE_PEER_ADDRESS']           = peeraddress
    
def createEnvironment(orgname): 
    os.environ['ORDERER_IP']="10.15.128.97"
    os.environ['ORDERER_HOSTNAME']="orderer.example.com" 
    os.environ['CHANNEL_NAME']="mychannel"
    os.environ['ORG_ID']=orgname
    os.environ['CORE_PEER_LOCALMSPID']="Org1MSP"
    os.environ['CORE_PEER_TLS_ROOTCERT_FILE']=crt1
    os.environ['CORE_PEER_MSPCONFIGPATH']=msp1
    os.environ['ORDERER_CA']=ordererpem
    os.environ['CORE_PEER_ADDRESS'] = "peer0.org1.example.com:7051"
    
def inform_existing_nodes():
    !/home/jovyan/work/scray/projects/invoice-hyperledger-fabric/scripts/inform_existing_nodes_local.sh $ORDERER_IP $CHANNEL_NAME $ORG_ID "/home/jovyan/work/fabric-samples/test-network" /tmp
    
def example_network_install_and_approve_cc():     
    !/home/jovyan/work/scray/projects/invoice-hyperledger-fabric/scripts/example_network_install_and_approve_cc.sh 10.14.128.38 10.15.128.97 /home/jovyan/work/fabric-samples/test-network    
    
def example_network_commit_cc():    
    os.environ['IP_CC_SERVICE']='10.14.128.38'
    #Host where the example network is running
    os.environ['IP_OF_EXAMPLE_NETWORK']='10.15.128.97'
    !/home/jovyan/work/scray/projects/invoice-hyperledger-fabric/scripts/example_network_commit_cc.sh $IP_CC_SERVICE $IP_OF_EXAMPLE_NETWORK /home/jovyan/work/fabric-samples/test-network   
    
def getCurrentConfig():
    !peer channel fetch config config_block.pb -o orderer.example.com:7050 -c $CHANNEL_NAME --tls --cafile $ORDERER_CA
    !configtxlator proto_decode --input config_block.pb --type common.Block > config_block.json
    !jq .data.data[0].payload.data.config config_block.json > config.json
    

In [ ]:
getCurrentConfig()


In [2]:
def getCrt(org='',domain='kubernetes.research.dev.seeburger.de',host='10.15.136.41:30080',user='scray', pwd='scray',channel='mychannel'):
    try:    
        output = subprocess.check_output(['curl', '--user',user + ':' + pwd, 
                                               'http://' + host + '/ca/' + channel + '/' + org + '-' + domain + '-ca.crt'])
        #return str(callProcess)[3:-3]
        return output.decode('ascii')
    except Exception as e:
        return str(e)    

def getCrtToLocal(org='',topath='/tmp',domain='kubernetes.research.dev.seeburger.de',host='10.15.136.41:30080',user='scray', pwd='scray',channel='mychannel'):
    try:    
        output = subprocess.check_output(['curl', '--user',user + ':' + pwd, 
                                               'http://' + host + '/ca/' + channel + '/' + org + '-' + domain + '-ca.crt', '-o', '/tmp/new.crt'])
        #return str(callProcess)[3:-3]
        return output.decode('ascii')
    except Exception as e:
        return str(e)   

def uploadFromLocal(host='10.15.136.41:30080',user='scray', pwd='scray',
                    local_path='./', local_name='' ,remote_path='upload'):
    try:    
        output = subprocess.check_output(['curl', '--user',user + ':' + pwd, 
                                          '-T', local_path + '/' + local_name, 
                                          'http://' + host + '/' + remote_path + '/' + local_name])
        return output.decode('ascii')
    except Exception as e:
        return str(e)   
    
def downloadToLocal(host='10.15.136.41:30080',user='scray', pwd='scray',
                    local_path='./', local_name='' ,remote_path='upload'):  
    try:    
        output = subprocess.check_output(['curl', '--user',user + ':' + pwd, 
                                               'http://' + host + '/' + remote_path + '/' + local_name, '-o', local_path + '/' + local_name])
        #return str(callProcess)[3:-3]
        return output.decode('ascii')
    except Exception as e:
        return str(e)     
    
def commit():
    try:    
        #crtnew = '\" + getCrt(org='klaus') + '\"
        getCrt(org='klaus')
        output = subprocess.check_output(['peer', 'lifecycle','chaincode','commit','-o',addr0,'--ordererTLSHostnameOverride',orderer,
                                          '--tls','--cafile',ordererpem,'--channelID','mychannel','--name','basic',
                                          '--peerAddresses',addr1, '--tlsRootCertFiles', crt1, 
                                          '--peerAddresses', addr2, '--tlsRootCertFiles', crt2, 
                                          '--peerAddresses', 'peer0.scray.org', '--tlsRootCertFiles', '/tmp/new.crt',
                                          #'--peerAddresses', 'kubernetes.research.dev.seeburger.de:32190', '--tlsRootCertFiles', '/tmp/new.crt',
                                          '--version', '1.0', '--sequence', '1','--tlsRootCertFiles'])
        return output.decode('ascii')
    except Exception as e:
        return str(e)
    
#commit()        

In [4]:
createEnvironment('bbc8')
!echo $ORDERER_IP $CHANNEL_NAME $ORG_ID

inform_existing_nodes()

10.15.128.97 mychannel bbc8
2021-01-01 18:24:42.330 UTC [channelCmd] InitCmdFactory -> INFO 001 Endorser and orderer connections initialized
2021-01-01 18:24:42.334 UTC [cli.common] readBlock -> INFO 002 Received block: 13
2021-01-01 18:24:42.334 UTC [channelCmd] fetch -> INFO 003 Retrieving last config block: 8
2021-01-01 18:24:42.339 UTC [cli.common] readBlock -> INFO 004 Received block: 8
<!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//EN">
<html><head>
<title>301 Moved Permanently</title>
</head><body>
<h1>Moved Permanently</h1>
<p>The document has moved <a href="http://10.15.136.41:30080/ca/">here</a>.</p>
</body></html>
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 11403  100 11403    0     0  1855k      0 --:--:-- --:--:-- --:--:-- 1855k
2021-01-01 18:24:42.722 UTC [channelCmd] InitCmdFactory -> INFO 001 Endorser and orderer connections initialized
2021-01-01 18:24:4

In [ ]:
createEnvironmentForPeer(localmspid=mspid1, tlsrootcert=crt1, mspconfigpath=msp1, peeraddress=addr1)
!peer channel signconfigtx -f org3_update_in_envelope.pb

createEnvironmentForPeer(localmspid=mspid2, tlsrootcert=crt2, mspconfigpath=msp2, peeraddress=addr2)
!peer channel signconfigtx -f org3_update_in_envelope.pb

In [ ]:
!configtxlator proto_decode --type='common.Envelope' --input=org3_update_in_envelope.pb --output=a.json

In [ ]:
os.environ['SHARED_FS_USER'] = "scray"
os.environ['SHARED_FS_PW']   = "scray"
os.environ['SHARED_FS_HOST'] = "10.15.136.41:30080"

#!curl --user $SHARED_FS_USER:$SHARED_FS_PW -X MKCOL http://$SHARED_FS_HOST/upload/
 

uploadFromLocal(local_path='./', local_name='org3_update_in_envelope.pb' ,remote_path='upload')

In [ ]:
downloadToLocal(local_path='./', local_name='org3_update_in_envelope.pb' ,remote_path='upload')

In [ ]:
!ls ./org3_update_in_envelope.pb
#!mv org3_update_in_envelope.pb org3_update_in_envelope1.pb

In [ ]:
#curl --user $SHARED_FS_USER:$SHARED_FS_PW -X MKCOL http://$SHARED_FS_HOST/newmemberrequests/
#curl --user $SHARED_FS_USER:$SHARED_FS_PW -X MKCOL http://$SHARED_FS_HOST/newmemberrequests/$CHANNEL_NAME
#curl --user $SHARED_FS_USER:$SHARED_FS_PW -X DELETE http://$SHARED_FS_HOST/newmemberrequests/$CHANNEL_NAME/${ORG_NAME}.json
#curl --user $SHARED_FS_USER:$SHARED_FS_PW -T organizations/peerOrganizations/$DOMAINE/${ORG_NAME}.json http://$SHARED_FS_HOST/newmemberrequests/$CHANNEL_NAME/${ORG_NAME}.json
    
# Upload CA
#curl --user $SHARED_FS_USER:$SHARED_FS_PW -X MKCOL http://$SHARED_FS_HOST/ca
#curl --user $SHARED_FS_USER:$SHARED_FS_PW -X MKCOL http://$SHARED_FS_HOST/ca/$CHANNEL_NAME
#curl --user $SHARED_FS_USER:$SHARED_FS_PW -T organizations/peerOrganizations/$DOMAINE/users/User1@$DOMAINE/tls/ca.crt http://$SHARED_FS_HOST/ca/$CHANNEL_NAME/$DOMAINE-ca.crt


In [ ]:
!peer channel update -f org3_update_in_envelope.pb -c $CHANNEL_NAME -o orderer.example.com:7050 --tls --cafile $ORDERER_CA


In [ ]:
!rm -f /tmp/new.crt
res = getCrtToLocal(org='tanja2')
#res = getCrt(org='lara2')
print(res)
!ls /tmp/*.crt
commit()

In [ ]:
!/home/jovyan/work/scray/projects/invoice-hyperledger-fabric/scripts/example_network_install_and_approve_cc.sh 10.14.128.38 10.15.128.97 /home/jovyan/work/fabric-samples/test-network

In [ ]:
peer lifecycle chaincode approveformyorg 
-o orderer.example.com:7050 
--ordererTLSHostnameOverride orderer.example.com 
--tls  
--cafile /tmp/tlsca.example.com-cert.pem 
--channelID $CHANNEL_ID 
--name basic 
--version 1.0 --package-id $PKGID --sequence 1

In [ ]:
#print(getCrt(org='klaus'))

getCrtToLocal(org='tanja2')

In [ ]:
getCrt(org='klaus')
!ls /tmp

In [ ]:
os.environ['IP_CC_SERVICE']='10.14.128.38'
#Host where the example network is running
os.environ['IP_OF_EXAMPLE_NETWORK']='10.15.128.97'

!/home/jovyan/work/scray/projects/invoice-hyperledger-fabric/scripts/example_network_commit_cc.sh $IP_CC_SERVICE $IP_OF_EXAMPLE_NETWORK /home/jovyan/work/fabric-samples/test-network

In [ ]:
!cat /home/jovyan/work/scray/projects/invoice-hyperledger-fabric/scripts/example_network_commit_cc.sh

# old

In [ ]:

!peer channel fetch config /tmp/config_block.pb -o orderer.example.com:7050 -c $CHANNEL_NAME --tls --cafile $ORDERER_CA > /dev/null
!configtxlator proto_decode --input /tmp/config_block.pb --type common.Block | jq .data.data[0].payload.data.config > /tmp/config.json   

In [ ]:
# Upload CA cert
os.environ['SHARED_FS_HOST']="10.15.136.41:30080"
os.environ['SHARED_FS_USER']="scray"
os.environ['SHARED_FS_PW']="scray"
#!apk add curl

os.environ['NEW_ORG_NAME']="OrgSally"

!curl --user $SHARED_FS_USER:$SHARED_FS_PW -X MKCOL http://$SHARED_FS_HOST/ca
!curl --user $SHARED_FS_USER:$SHARED_FS_PW -T /home/jovyan/work/fabric-samples/test-network/organizations/ordererOrganizations/example.com/orderers/orderer.example.com/msp/tlscacerts/tlsca.example.com-cert.pem http://$SHARED_FS_HOST/ca/tlsca.example.com-cert.pem

# Get configuration of new peer
#curl --user 'scray:scray' http://${SHARED_FS_HOST}/newmemberrequests/$CHANNEL_NAME/${NEW_ORG_NAME}.json > new_member_org.json
!curl --user $SHARED_FS_USER:$SHARED_FS_PW  http://${SHARED_FS_HOST}/newmemberrequests/$CHANNEL_NAME/${NEW_ORG_NAME}.json > /tmp/new_member_org.json 
        
# Add org3 data to existing config
!jq -s ".[0] "*" {\"channel_group\":{\"groups\":{\"Application\":{\"groups\": {\"${NEW_ORG_NAME}MSP\":.[1]}}}}}" /tmp/config.json /tmp/new_member_org.json > /tmp/modified_config.json

!configtxlator proto_encode --input /tmp/config.json --type common.Config --output /tmp/config.pb
!configtxlator proto_encode --input /tmp/modified_config.json --type common.Config --output /tmp/modified_config.pb
!configtxlator compute_update --channel_id $CHANNEL_NAME --original /tmp/config.pb --updated /tmp/modified_config.pb --output /tmp/org3_update.pb
!configtxlator proto_decode --input /tmp/org3_update.pb --type common.ConfigUpdate | jq . > /tmp/org3_update.json
!echo '{"payload":{"header":{"channel_header":{"channel_id":"'$CHANNEL_NAME'", "type":2}},"data":{"config_update":'$(cat /tmp/org3_update.json)'}}}' | jq . > /tmp/org3_update_in_envelope.json
!configtxlator proto_encode --input /tmp/org3_update_in_envelope.json --type common.Envelope --output /tmp/org3_update_in_envelope.pb        

sign and update

In [ ]:
!peer channel signconfigtx -f /tmp/org3_update_in_envelope.pb

os.environ['CORE_PEER_LOCALMSPID']="Org2MSP"
os.environ['CORE_PEER_TLS_ROOTCERT_FILE']=crt2
os.environ['CORE_PEER_MSPCONFIGPATH']=msp2
os.environ['ORDERER_CA']=ordererpem
os.environ['CORE_PEER_ADDRESS'] = addr2

!peer channel update -f /tmp/org3_update_in_envelope.pb -c $CHANNEL_NAME -o orderer.example.com:7050 --tls --cafile $ORDERER_CA

# network_install_and_approve_cc (3 nodes)

In [ ]:
#!pwd
#!ls /home/jovyan/work/scray/projects/invoice-hyperledger-fabric/scripts/example_network_install_and_approve_cc.sh
!cd /tmp
!/home/jovyan/work/scray/projects/invoice-hyperledger-fabric/scripts/example_network_install_and_approve_cc.sh 10.14.128.38 10.15.128.97 /home/jovyan/work/fabric-samples/test-network

# commit

In [ ]:
os.environ['IP_CC_SERVICE']='10.14.128.38'
#Host where the example network is running
os.environ['IP_OF_EXAMPLE_NETWORK']='10.15.128.97'

!/home/jovyan/work/scray/projects/invoice-hyperledger-fabric/scripts/example_network_commit_cc.sh $IP_CC_SERVICE $IP_OF_EXAMPLE_NETWORK /home/jovyan/work/fabric-samples/test-network

# check network (approved chaincode)

In [ ]:
!peer lifecycle chaincode querycommitted --channelID mychannel --name basic --output json

#!peer lifecycle chaincode checkcommitreadiness --channelID mychannel --name basic --output json --version  1.0 --sequence 2

#!peer lifecycle chaincode --channelID mychannel
#!discover --configFile conf.yaml peers --channel mychannel  --server peer0.org1.example.com:7051

#!discover --configFile config.json peers --channel mychannel  --server peer0.org1.example.com:7051

In [ ]:
!peer lifecycle chaincode querycommitted --channelID mychannel --name scray-invoice-example --output json
!peer lifecycle chaincode queryinstalled

In [ ]:
#!peer chaincode query -C mychannel -n scray-invoice-example -c '{"Args":["get","name"]}'
#./network.sh createChannel -c channelone
!peer channel list
!peer lifecycle chaincode querycommitted --channelID channelone --name scray-invoice-example --output json

In [ ]:
!env | fgrep MSP

# Test

In [ ]:
def readAsset(asset='asset1'):
    callopt= '{\"function\":\"ReadAsset\",\"Args\":[' + '\"' + asset + '\"' + ']}'
    try:    
        callProcess  = subprocess.check_output(['peer', 'chaincode','query', '-C', 'mychannel', '-n', 'basic', '-c', callopt])
        return json.loads(str(callProcess)[2:-3])
    except Exception as e:
        return str(e)
    
readAsset('asset1')    

In [ ]:
readAsset('asset2')  

In [ ]:
readAsset('asset4')  

In [ ]:
!peer chaincode query -C mychannel -n basic -c '{"function":"GetAllAssets","Args":[]}'

# REST

In [ ]:
!peer channel fetch config config_block.pb -o orderer.example.com:7050 -c $CHANNEL_NAME --tls --cafile $ORDERER_CA
!configtxlator proto_decode --input config_block.pb --type common.Block > config_block.json
    

In [ ]:
!peer channel fetch config /tmp/config_block.pb -o orderer.example.com:7050 -c $CHANNEL_NAME --tls --cafile $ORDERER_CA > /dev/null  

In [ ]:
#peer channel fetch config config_block.pb -o orderer.example.com:7050 -c $CHANNEL_NAME --tls --cafile $ORDERER_CA
import sys    
def channelFetch(channelName,ordererpem):
    #rint(channelName,ordererpem)
    try:    
        output = subprocess.check_output(['peer', 'channel','fetch', 'config', 'config_block.pb', '-o', 'orderer.example.com:7050', '-c',channelName, '--tls', 
                                               '--cafile', ordererpem], stderr=subprocess.STDOUT, shell=True, timeout=3,universal_newlines=True)
        #output = subprocess.check_output(['peer', 'channel','fetch', 'config', 'config_block.pb', '-o', 'orderer.example.com:7050', '-c',channelName, '--tls', 'cafile', ordererpem], stderr=subprocess.STDOUT, shell=True, timeout=3,universal_newlines=True)
        #output = subprocess.check_output(['peer', 'channel','fetch','newest', 'config', '/tmp/config_block.pb', '-o', 'orderer.example.com:7050', '-c',channelName, '--tls', '--cafile', ordererpem], stderr=subprocess.STDOUT)
        
    #xcept Exception as e:
    #   print('exception' + str(e))
    #   return None
    except subprocess.CalledProcessError as exc:
        print("Status : FAIL", exc.returncode, exc.output)
    else:
        print("Output: \n{}\n".format(output))
    
#channelFetch('mychannel',ordererpem)    

#!cd /home/jovyan/work/
#rm -f config_block.pb
channelFetch('mychannel',ordererpem)
